In [1]:
# ROSCORE must be running!

import numpy as np
from sensor_msgs.msg import PointCloud2, PointField
import rospy
import math
import sys
from sensor_msgs.msg import PointCloud2
import std_msgs.msg
import sensor_msgs.point_cloud2 as pc2
import rospy
from IPython.display import clear_output
import rosbag

path = "/media/juli/98F29C83F29C67721/SemesterProject/data/1_training/dual_topic/demo/single_intensity"
file_names = ["/1-dust_labeled_spaces", "/2-dust_labeled_spaces", "/3-dust_labeled_spaces",
              "/4-dust_labeled_spaces", "/5-dust_labeled_spaces", "/6-dust_labeled_spaces",
              "/7-dust_labeled_spaces", "/8-dust_labeled_spaces", "/9-smoke_labeled_spaces",
              "/10-smoke_labeled_spaces", "/11-smoke_labeled_spaces", "/12-smoke_labeled_spaces", 
              "/13-smoke_labeled_spaces", "/14-smoke_labeled_spaces", "/15-smoke_labeled_spaces", 
              "/16-smoke_labeled_spaces", "/17-smoke_labeled_spaces","/18-smoke_labeled_spaces", 
              "/19-smoke_labeled_spaces"]
file_names = ["/1-dust_labeled_spaces_img_predicted_pcl", "/20-smoke_labeled_img_predicted_pcl"]

In [2]:
# Publish Labeled Dual Pointcloud
if __name__ == '__main__':
    for file_name in file_names:
        # Choose correct file
        load_file = path + file_name
        frames = np.load(load_file + ".npy")
        # Prepare rosbag
        outbag = rosbag.Bag(load_file + ".bag", 'w')
        # Prepare topic
        cloud_msg = PointCloud2() 
        cloud_msg.fields = [
            PointField('x', 0, PointField.FLOAT32, 1),
            PointField('y', 4, PointField.FLOAT32, 1),
            PointField('z', 8, PointField.FLOAT32, 1),
            PointField('intensity', 12, PointField.FLOAT32, 1),
            PointField('r', 16, PointField.FLOAT32, 1),
            PointField('l_none', 20, PointField.FLOAT32, 1),
            PointField('l_dust', 24, PointField.FLOAT32, 1),
            PointField('l_smoke', 28, PointField.FLOAT32, 1)]
        # Information to terminal
        rospy.loginfo("Initializing pc2 rosbag writer...")
        # Give time to roscore to make the connections
        rospy.sleep(1.)
        # Create writer
        rospy.init_node('dual_pc2_writer', anonymous=True)
        # Header
        header = std_msgs.msg.Header()
        header.frame_id = 'map'

        # Publish    
        rospy.loginfo("happily publishing sample pointcloud.. !")
        for i, frame in enumerate(frames):
            if i == 0:
                header.stamp = rospy.Time.now()
            else:
                header.stamp = header.stamp + rospy.Duration(0.1)
            # Add labels
            rospy.loginfo("Frame: %s", i)
            clear_output(wait=True)
            
            points = np.asarray(frame[:, 5:10])
            points_dual = np.asarray(frame[:, :5])
            labels = np.asarray(frame[:, len(frame[0,:])-3:len(frame[0,:])])
            pointcloud = np.concatenate((points, labels), axis = 1)
            pointcloud_dual = np.concatenate((points_dual, labels), axis = 1)
            # Create pcl from points
            scaled_polygon_pcl = pc2.create_cloud(header, cloud_msg.fields, pointcloud)
            scaled_polygon_pcl_dual = pc2.create_cloud(header, cloud_msg.fields, pointcloud_dual)
            outbag.write("/velodyne_points", scaled_polygon_pcl, header.stamp)
            outbag.write("/velodyne_points_dual", scaled_polygon_pcl_dual, header.stamp)
            #rate.sleep()
        outbag.close()

[INFO] [1547647907.188477]: Frame: 765
